# Bigram Model

In [1]:
import torch
import torch.nn as nn 
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from  sklearn import linear_model
%matplotlib inline

# You might prefer: Karpathy's nanoGPT
# train.py : More complicated version of training process (like in makemore)
# model.py : Essentially identical to the model in gpt.ipynb

In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [3]:
# READ DATA
with open ('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    

In [ ]:
# EXPLORE DATA
print(text[:1000])

In [ ]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)

In [6]:
# ENCODE DATA
# Most basic one-hot encoding.
# What is a tokenizer? This is a tokenizer:
stoi = {s:i for i,s in enumerate(chars)}
itos = {i:s for s,i in stoi.items()}
def encode(s: str) -> torch.tensor:
    return torch.tensor([stoi[c] for c in s])
def decode(s: torch.tensor) -> str:
    return ''.join([itos[int(c)] for c in list(s)])

In [ ]:
print(encode('hello, there'))
print(decode(encode('hello, there')))

In [8]:
# Google uses : SentencePiece
# tiktoken used for gpt2 (this is what we build next time?)
# fast BPE tokenizer

In [ ]:
# SPLIT DATA
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape)
print(data[:10])
n = int(0.8*len(text))
n2 = int(0.9*len(text))
train_data = data[:n]
val_data = data[n:n2]
test_data = data[n2:]



In [10]:
# HELPER FUNCTIONS FOR TRAINING

# x = train_data[:block_size]
# y = train_data[1:block_size+1]

# BATCHING
# This is pretty similar to how batches were processed in makemore
def get_batch(split, block_size, batch_size, device='cuda'):
    data = train_data if split == 'train' else val_data if  split == 'val' else test_data 
    ix = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    # New, adding to put data on the GPU:
    x, y  = x.to(device), y.to(device)
    return x, y

# xb, yb = get_batch('train', block_size, batch_size)
# print(xb.shape, yb.shape)
# print(decode(xb[0]), decode(yb[0]))

# ESTIMATE LOSS
# Better estimate than just using the loss on the last batch—
# get a less noisy result by averaging over multiple batches.
@torch.no_grad()
def estimate_loss(model, block_size, batch_size, eval_iters,  device='cuda'):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for i in range(eval_iters):
            x, y = get_batch(split, block_size, batch_size, device=device)
            logits, loss = model(x, y)
            loss = F.cross_entropy(yhat.view(-1, vocab_size), y.view(-1))
            losses[i] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


In [11]:
# CONSTRUCT MODEL
# Establishing the structure for a torch model by revisiting bigram
class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):
        # idx, targets are (B, T)
        logits = self.token_embedding_table(idx) # (B, T, C)
        
        if targets is not None: 
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        else:
            loss = None
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T)
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :] # Becomes (B, C)
            probs = F.softmax(logits, dim=-1) # (B, C)
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            idx = torch.cat((idx, idx_next), dim=1)

        return idx


In [12]:
# Hyperparameters
block_size = 8 #T,  = context window? If context window is longer, need to truncate context window for transformer to understand how to predict
batch_size = 64 #B,  how many blocks to process at once
vocab_size = 65 # C, in Karpathy's shorthand
learning_rate = 1e-3
train_steps = 10000
device = 'cuda'
eval_iters = 500

if not torch.cuda.is_available():
    print('**\n**\n**\n**ERROR: CUDA ISNT RUNNING YET. CODE BELOW WILL FAIL**\n**\n**\n**')

In [13]:
model = BigramLanguageModel(vocab_size)
m = model.to(device)

In [14]:
# How do we use Torch to train?
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
# A more advanced and modern training method:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# TRAINING LOOP
for steps in range(train_steps):
    xb, yb = get_batch('train', block_size, batch_size, device=device)
    logits, loss = model(xb, yb)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if steps % 1000 == 0:
        print(f'Step {steps}, Loss {loss.item()}')

In [ ]:
# EVALUATE MODEL


In [ ]:
# GENERATE TEXT
context = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(model.generate(context,500)[0])) # (slightly different because I encode/decode in torch)

In [ ]:
torch.manual_seed(33396887) # deeznuts
torch.cuda.is_available()